Task from NewProLab BigData Course.

Classification of movie reviews from IMDB into positive and negative on the Kaggle platform

We need to determine the positivity/negativity of movie reviews by using reviews of other movies for training.

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Import the pandas package, then use the "read_csv" function to read
# the labeled training data
      
train = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip",
                    header=0,
                    delimiter="\t",
                    quoting=3)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip", 
                   header=0,
                   delimiter="\t",
                   quoting=3 )



In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
print(train["review"][0])

In [ ]:
import re
import nltk

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)  # 'r([^\w]|[+])'
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence,
                                                                           remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences

In [ ]:
# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list

print ("Cleaning and parsing the training set movie reviews...\n")
for i in range( 0, len(train["review"])):
    clean_train_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(train["review"][i], True)))

In [ ]:
clean_train_reviews[0]

In [ ]:
len(clean_train_reviews)


CountVectorization

The Bag of Words model learns a vocabulary from all of the documents, then models each document by counting the number of times each word appears. For example, consider the following two sentences:

Sentence 1: "The cat sat on the hat"

Sentence 2: "The dog ate the cat and the hat"

From these two sentences, our vocabulary is as follows:

{ the, cat, sat, on, hat, dog, ate, and }

To get our bags of words, we count the number of times each word occurs in each sentence. In Sentence 1, "the" appears twice, and "cat", "sat", "on", and "hat" each appear once, so the feature vector for Sentence 1 is:

{ the, cat, sat, on, hat, dog, ate, and }

Sentence 1: { 2, 1, 1, 1, 1, 0, 0, 0 }

Similarly, the features for Sentence 2 are: { 3, 1, 0, 0, 1, 1, 1, 1}

In the IMDB data, we have a very large number of reviews, which will give us a large vocabulary. To limit the size of the feature vectors, we should choose some maximum vocabulary size. Below, we use the 5000 most frequent words (remembering that stop words have already been removed).


In [ ]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.
countVectorizer = CountVectorizer(analyzer = "word",   \
                         tokenizer = None,    \
                         preprocessor = None, \
                         stop_words = None,   \
                         max_features = 5000)

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of
# strings.
train_data_features = countVectorizer.fit_transform(clean_train_reviews)

In [ ]:
print(countVectorizer.get_feature_names())

In [ ]:
type(train_data_features)

In [ ]:
# Numpy arrays are easy to work with, so convert the result to an
# array
np.asarray(train_data_features)

In [ ]:
print ("Training the random forest (this may take a while)...")


# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100)

# Fit the forest to the training set, using the bag of words as
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

In [ ]:
# Testing - we pick only the first 2 test reviews alone to save time


# Create an empty list and append the clean reviews one by one
clean_test_reviews = []

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,len(test)):
    clean_test_reviews.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(test["review"][i], True)))

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = countVectorizer.transform(clean_test_reviews)
np.asarray(test_data_features)

# Use the random forest to make sentiment label predictions
print ("Predicting test labels...\n")
result = forest.predict(test_data_features)

# Write the test results 
print("*****manual verification*******")
print(test["review"][0])
print(result[0])
print(test["review"][1])
print(result[1])

In [ ]:
test["id"]

In [ ]:
result

In [ ]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test.id.apply(lambda x: x.replace('"', '')), "sentiment":result} )

In [ ]:
output.head()

In [ ]:
test.head()

In [ ]:
# Use pandas to write the comma-separated output file
output.to_csv('./Submission.csv', index=False)
print ("Wrote results to Submission.csv")

In [ ]:
type(output)

In [ ]:
output.head()

In [ ]:
kaggle competitions submit -c word2vec-nlp-tutorial -f submission.csv -m "Message"